In [ ]:
    from load_feat_pd import *
    import sys
    import json
   

    np.set_printoptions(precision=2)

    config_filepath = 'config_test.json'

    with open(config_filepath, "r") as json_file:
        config_data = json.load(json_file)

    feats2level = config_data["feats2level"]
    allfeats_batch1 = config_data["allfeats_batch1"]
    allfeats_batch2 = config_data["allfeats_batch2"]
    features_to_load = config_data["features_to_load"]
    merged_data_pkl = config_data["merged_data_pkl"]
    base_folder = config_data["base_folder"]
    list_not_load = config_data["list_not_load"]
    # for feat in allfeats:
    # for feat in allfeats:
    

    #metadata = load_feat(base_folder_path, feature_name=featname)
    # load_feat(base_folder_path, feature_name='contrast', threeD=True)
    for featname in features_to_load:
        metadata = load_feat_single(base_folder, feature_name=featname)
        df = pd.DataFrame(metadata)
        print(df)
        if feats2level[featname] == 'utt':
            # Compute mean and standard deviation of 'value' for each subject_id
            stats_df = df.groupby("subject_id")["value"].agg(value_mean="mean", value_std="std").reset_index()

            # Identify columns to keep (all except 'value', 'filename', and 'item')
            other_cols = [col for col in df.columns if col not in ["value", "filename", "item"]]

            # Get one representative row per subject (assuming these columns are identical within each subject)
            other_df = df.drop_duplicates("subject_id")[other_cols]

            # Merge the aggregated stats with the other columns based on subject_id
            result_df = pd.merge(other_df, stats_df, on="subject_id", how="left")

            # result_df now contains the group-level statistics and the retained columns
            print(result_df.head())
            print(result_df)

        elif feats2level[featname] == 'frame':
            continue
        elif feats2level[featname] == '3d':
            continue
    

Processing folder: /data/storage1t/Turntaking/allnew_merged_wavs-features
Found 3083 npy files
0 files with all 0 values
      experiment group_id                value subject_id  \
0     exp_single       22   0.1441852717891061       2208   
1     exp_single       22  0.12082249209179198       2208   
2     exp_single       22  0.15981639152969385       2208   
3     exp_single       22   0.1767153919626867       2208   
4     exp_single       22  0.13862901245690004       2208   
...          ...      ...                  ...        ...   
2479  exp_single       22   0.0912861695355453       2206   
2480  exp_single       22  0.09797609118671156       2206   
2481  exp_single       22  0.11071385095707802       2206   
2482  exp_single       22  0.10902973850457079       2206   
2483  exp_single       22  0.09149174424157198       2206   

                             filename    item age gender moca education romp  \
0      subj-2208_merged_0.wav_shimmer  merged  66      M   29     